## Randomness and the tragedy of sparsity, or how to cheat chance

In [6]:
import numpy as np
import os
ff = []
lat = []

In [7]:
seed = 7
np.random.seed(seed)
recording_filename = str(np.random.randint(100000, 1000000))

if os.path.isfile(recording_filename + ".npz"):
    with np.load(recording_filename + ".npz")as init_data:
        print init_data.keys()
        ff = init_data['ff_conn']
        lat = init_data['lat_conn']

['lat_w', 'lat_conn', 's', 'ff_w', 'ff_conn']


In [11]:
conn_ff =  np.count_nonzero(ff) / float(ff.size) * 100
print "Connectivity of ff", conn_ff, "%"
print ",as such, the sparsity of it is", 100. - conn_ff, "%"

Connectivity of ff 6.25 %
,as such, the sparsity of it is 93.75 %


In [12]:
conn_lat =  np.count_nonzero(lat) / float(lat.size) * 100
print "Connectivity of lat", conn_lat, "%"
print ",as such, the sparsity of it is", 100. - conn_lat, "%"

Connectivity of lat 6.25 %
,as such, the sparsity of it is 93.75 %


Let's focus on Feedforward projections

In [14]:
print "There are", np.count_nonzero(ff), "feeedforward projections, out of ",  ff.size,"possible projections"

There are 4096 feeedforward projections, out of  65536 possible projections


In [19]:
print "As such, the probability of randomly choosing a formed synapse to consider for elimination \
(using a uniform random process) is ", float(np.count_nonzero(ff)) / ff.size

As such, the probability of randomly choosing a formed synapse to consider for elimination (using a uniform random process) is  0.0625


In [26]:
def find_projection():
    global ff, lat
    # First, choose a type of projection (FF or LAT)
    projection_index = np.random.randint(0, 2)
    # Second, choose a postsynaptic neuron from that projection
    postsynaptic_neuron_index = np.random.randint(0, 256)

    potential_pre =  np.random.randint(0, 256)
    _2d_to_1d_arithmetic = potential_pre * 256 + postsynaptic_neuron_index
    #         _2d_to_1d_arithmetic = pre_x * N + pre_y
    # Third, check if the synapse exists or not and follow the appropriate rule
    if  projection_index==0 and ff[_2d_to_1d_arithmetic] or projection_index==1 and lat[_2d_to_1d_arithmetic]:
        return _2d_to_1d_arithmetic
    return None

In [31]:
count = 0
for _ in range(20000):
    if find_projection():
        count += 1

In [32]:
print count

1271


In [33]:
print count / 20000.

0.06355


This seems to match up with our expectation. But why did I not get these results?

Because someone **FORGOT** what potential pre-synaptic neuron meant and would only return indices corresponding to neurons which were **NOT CONNECTED**, meaning I could **NEVER** eliminate a synapse because I would only look at connections that should have been formed. GG

In [30]:
def potential_presynaptic_neuron(self, projection, postsynaptic_index):
        potential_neurons = \
            np.nonzero(np.invert(projection.synapse_connected.reshape(self.N, self.N)[:, postsynaptic_index]))[0]
        if len(potential_neurons) == 0:
            return None
        random_index = potential_neurons[np.random.randint(0, len(potential_neurons))]
        return random_index